In [2]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re
import time
from tqdm import tqdm, trange
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [11]:
class DBHelper:

    def __init__(self, db_name = "recipe.db"):
        self.dbname = f"sqlite3:/{db_name}"
        self.conn = sqlite3.connect(db_name)

    #for link, recipe, ingredient and category
    def create_table_simple(self, table_name, params):
        column = ""
        for i in range(len(params) - 1):
            column += params[i] + " TEXT, "
        column += params[len(params) - 1] + " TEXT "

        new_table = f"CREATE TABLE IF NOT EXISTS {table_name} (id INTEGER PRIMARY KEY AUTOINCREMENT , {column})"
        #print(new_table)
        try:
            #print(new_table)
            self.conn.execute(new_table)
            self.conn.commit()
        except sqlite3.Error as err:
            #print('Sql error: %s' % (' '.join(err.args)))
            self.conn.rollback()

    #for calories and recipe_ingredients
    def create_table(self, table_name, primary_key, foreign_table_name, foreign_col, another_foreign_col, params, auto = False):
        column = ""
        for i in range(len(params) - 1):
            column += params[i] + " TEXT, "
        column += params[len(params)-1] + " TEXT "

        foreign_key = f"FOREIGN KEY ({foreign_col})  REFERENCES {foreign_table_name} ({another_foreign_col})"

        new_table = f"CREATE TABLE IF NOT EXISTS {table_name} ({primary_key} INTEGER PRIMARY KEY {('', 'AUTOINCREMENT')[auto == True]} , {column}, {foreign_key})"

        try:
            #print(new_table)
            self.conn.execute(new_table)
            self.conn.commit()
        except sqlite3.Error as err:
            #print('Sql error: %s' % (' '.join(err.args)))
            self.conn.rollback()

    #id autoincrement
    def generic_create_table(self, table_name, primary_keys, foreign_keys, references, unique_args, other_args, auto):

        new_table = ""

        begin = f"CREATE TABLE IF NOT EXISTS {table_name} ("

        primary = ""
        column = " "

        if len(primary_keys) > 1:
            for i in range(len(primary_keys) - 1):
                primary += primary_keys[i][0] + ", "
                column += primary_keys[i][0] + f" {primary_keys[i][1]}, "

            primary += primary_keys[len(primary_keys) - 1][0]
            column += primary_keys[len(primary_keys) - 1][0] + f" {primary_keys[len(primary_keys) - 1][1]}, "

            new_table += f"PRIMARY KEY ({primary})"
        else:
            begin += f"{primary_keys[0][0]} {primary_keys[0][1]} PRIMARY KEY {('', 'AUTOINCREMENT')[auto]}"
            column += ", " + column

        if foreign_keys is not None and len(foreign_keys) != 0:
            reference = ""
            for i in range(len(references) - 1):
                reference += f"FOREIGN KEY ({foreign_keys[i]}) REFERENCES {references[i][0]}({references[i][1]}), "
                column += foreign_keys[i] + " TEXT, "
            reference += f"FOREIGN KEY ({foreign_keys[len(foreign_keys) - 1]}) REFERENCES" \
                         f" {references[len(references) - 1][0]}({references[len(references) - 1][1]})"
            column += foreign_keys[len(foreign_keys) - 1] + " TEXT, "

            new_table += f", {reference}"

        if unique_args is not None and len(unique_args) != 0:
            unique = ""
            for i in range(len(unique_args) - 1):
                unique += unique_args[i] + ", "
                column += unique_args[i] + " TEXT, "
            unique += unique_args[len(unique_args) - 1]
            column += unique_args[len(unique_args) - 1] + " TEXT, "

            new_table += f", UNIQUE ({unique})"

        if other_args is not None and len(other_args) != 0:
            for i in range(len(other_args) - 1):
                column += other_args[i] + " TEXT, "
            column += other_args[len(other_args)-1] + " TEXT, "

        new_table = begin + (column, column[:-2])[len(primary_keys) == 1] + new_table + ")"

        try:
            print(new_table)
            self.conn.execute(new_table)
            #print("Выполнилось")
            self.conn.commit()
        except sqlite3.Error as err:
            #print('Sql error: %s' % (' '.join(err.args)))
            self.conn.rollback()

    def get_unique(self, table_name, name, unique_columns):
        column = ""
        for i in range(len(unique_columns) - 1):
            column += unique_columns[i] + ", "
        column += unique_columns[len(unique_columns)-1]

        try:
            #print("Unique")
            self.conn.execute(f"CREATE UNIQUE INDEX {name} ON {table_name}({column})")
            #print("Выполнилось")
            self.conn.commit()
        except sqlite3.Error as err:
            #print('Sql error: %s' % (' '.join(err.args)))
            self.conn.rollback()

    def insert_check(self, table_name, primary_key, primary_value, args):
        arguments = f"'{primary_value}', '"
        for i in range(len(args) - 1):
            arguments += str(args[i]) + "', '"
        arguments += str(args[len(args) - 1]) + "'"
        #print(arguments)

        new_insert = f"INSERT INTO {table_name} SELECT * FROM (SELECT {arguments}) AS tmp " \
                     f"WHERE NOT EXISTS ( SELECT {primary_key} FROM {table_name} WHERE {primary_key} = '{primary_value}'" \
                     f") LIMIT 1"
        try:
            #print(new_insert)
            self.conn.execute(new_insert)
            #print("Выполнилось")
            self.conn.commit()
        except sqlite3.Error as err:
            #print('Sql error: %s' % (' '.join(err.args)))
            self.conn.rollback()

    def insert(self, table_name, column_args, args):
        arguments = "'"
        for i in range(len(args) - 1):
            arguments += str(args[i]) + "', '"
        arguments += str(args[len(args) - 1]) + "'"
        #print(arguments)

        columns = ""
        for i in range(len(column_args) - 1):
            columns += str(column_args[i]) + ", "
        columns += str(column_args[len(column_args) - 1])

        new_insert = f"INSERT INTO {table_name} ({columns}) VALUES ({arguments})"
        #print(new_insert)

        try:
            print(new_insert)
            self.conn.execute(new_insert)
            print("Выполнилось")
            self.conn.commit()
            return True
        except sqlite3.Error as err:
            print('Sql error: %s' % (' '.join(err.args)))
            self.conn.rollback()
            return False

    def print_info(self, table_name):
        new_get = f"SELECT * FROM {table_name}"
        cursor = self.conn.execute(new_get)

        #print("Выполнилось")

        return [row for row in cursor]

    def get(self, table_name, column_args, args):
        query = ""
        for i in range(len(column_args) - 1):
            query += column_args[i] + " = '" + args[i] + "' AND "
        query += column_args[len(column_args) - 1] + " = '" + args[len(column_args) - 1] + "'"

        new_get = f"SELECT * FROM {table_name} WHERE {query}"
        #print(new_get)

        cursor = self.conn.execute(new_get)

        return [row for row in cursor]
    
    def get_image(self, table_name, column_args, args):
        query = ""
        for i in range(len(column_args) - 1):
            query += column_args[i] + " = '" + args[i] + "' AND "
        query += column_args[len(column_args) - 1] + " = '" + args[len(column_args) - 1] + "'"

        new_get = f"SELECT * FROM {table_name} WHERE {query}"
        #print(new_get)

        cursor = self.conn.execute(new_get)

        return [row for row in cursor]

    def rollback(self):
        self.conn.rollback()

    #def update(self, flat_id, updates):


    #def delete(self, flat_id):

In [69]:
class Crawler:        
    def get_html(self, url):
        pass

In [70]:
class CrawlerPovar(Crawler):
    def get_html(self, url):   
        '''
        chrome_options = Options()
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-popup-blocking")
        chrome_options.add_argument("--headless=new")

        self.browser = webdriver.Chrome(options=chrome_options)
        self.browser.get(url)

        wait = WebDriverWait(self.browser, 10)
        self.browser.implicitly_wait(30)

        soup_card = BeautifulSoup(self.browser.page_source, 'html.parser')
        '''
        #BeautifulSoup(requests.get(url).text, 'html.parser')
        return BeautifulSoup(requests.get(url).text, 'html.parser')

In [71]:
class Parser:

    def __init__(self):
        self.helper = DBHelper()

    def get_cards(self):
        pass

    def __get_ingredients(self):
        pass

    def __get_calories(self):
        pass

    def __get_manual(self):
        pass

    def __get_category(self):
        pass

    def get_recipe(self):
        pass

In [72]:
class ParserPovar(Parser):

    # def get_info(self, link, j):  
    #    if self.html.find('h1', itemprop="name") != None:
    #        photo = self.html.find('img', itemprop="image").get('src')
    #        j += 1
    #        DBHelper().insert("link", "id", [str(j), link, photo])
    #    return j

    def get_ingredients(self, html, id):
        ingredients = []
        html = html.find('div', class_="ingredients-bl")
        for ing in html.findAll('li', itemprop="recipeIngredient"):  
            ingredients = ing.findAll('span')
            name = ingredients[0].text
            count = ingredients[len(ingredients) - 1].text
            self.helper.insert('ingredient', ['name_ingr'], [str(name).replace("'", "''")])
            ingr_ = self.helper.get('ingredient', ['name_ingr'], [str(name).replace("'", "''")])
            self.helper.insert('recipe_ingredients',
                          ['recipe_id', 'ingr_id', 'count'],
                          [id, ingr_[0][0],
                           count])
            #print(name, count)

        return ingredients
       
    def get_manual(self, html, id):
        manual = ''
        html = html.find('ul', itemprop="recipeInstructions")
        for rec in html.findAll('li', class_="cooking-bl"):
            manual += '' + rec.find('p').text
        #print(manual)
        if manual == '':
            k = 1 / 0
        return manual

    def get_calories(self, html, id):
        html = html.find('div', itemprop="nutrition")
        calories_ = html.find('strong', itemprop="calories").text
        protein_ = html.find('strong', itemprop="proteinContent").text
        fat_ = html.find('strong', itemprop="fatContent").text
        carbohydrate_ = html.find('strong', itemprop="carbohydrateContent").text  
        calories = " ".join(calories_.split())
        protein = " ".join(protein_.split())
        fat = " ".join(fat_.split())
        carbohydrate = " ".join(carbohydrate_.split())
        all = [calories, protein, fat, carbohydrate]
        #print(calories, protein, fat, carbohydrate)
        self.helper.insert('calories', ['recipe_id', 'calories', 'proteins', 'fats', 'carbohydrates'], [id,
                                                  calories,
                                                  protein,
                                                  fat,
                                                  carbohydrate])
        return all
    
    def check_ingredients(self, html):
        html.find('div', class_="ingredients-bl")

    def get_category(self, html, id):
        category = []
        flag = False
        i = 0
        html = html.find('div', class_="article-breadcrumbs")
        for cat in html.findAll('span', itemprop="recipeCategory"):
            cate = " ".join(cat.find('a').text.split())
            category.append(cate)
            #print(category[i])
            #self.helper.insert('category', ['name_cat'], [category[i]])
            a = self.helper.get('category', ['name_cat'], [category[i]])
            if a != None and len(a) != 0:
                flag = True
                self.helper.insert('recipe_categories', ['recipe_id', 'cat_id'],
                            [id, a[0][0]])
            i += 1
        if not flag:
            self.helper.insert('category', ['name_cat'], [category[0]])    
            self.helper.insert('recipe_categories', ['recipe_id', 'cat_id'],
                            [id, self.helper.get('category', ['name_cat'], 
                                                 [category[0]])[0][0]])
        return category

    def get_info(self, html, id, link):  
        try:
            html = html.find('article', class_="item-bl item-about")
            self.check_ingredients(html)
            manual = self.get_manual(html, id)
            name_dish = html.find('h1', itemprop="name").text
            cook_time = html.find('time', itemprop="totalTime").text
            count_portion = html.find('span', itemprop="recipeYield").text
            photo = html.find('img', itemprop="image").get('src')
            flag = self.helper.insert('recipe', ['id', 'name_dish', 'count_portion', 'cook_time', 'manual'],
                            [id, name_dish, count_portion, cook_time, manual])
            self.helper.insert('link', ['id', 'link', 'photo'],
                            [id, link, photo])
            if flag:
                category = self.get_category(html, id)
                energy = self.get_calories(html, id)
                ingredients = self.get_ingredients(html, id) 
            return True
        except:
            return False


In [73]:
class Recipe:
    def __init__(self, name_dish, count_portion, cook_time, manual):
        self.name = name_dish
        self.count_portion = count_portion
        self.cook_time = cook_time
        self.manual = manual

In [74]:
class Category:
    def __init__(self, name_cat):
        self.name = name_cat

In [75]:
class Ingredient:
    def __init__(self, name_ingr):
        self.name = name_ingr

In [76]:
class Calories: #можно взять по айдишнику рецепта
    def __init__(self, calories, proteins, fats, carbohydrates):
        self.calories = calories
        self.proteins = proteins
        self.fats = fats
        self.carbohydrates = carbohydrates

In [77]:
class Link:
    def __init__(self, link, photo):
        self.link = link
        self.photo = photo

In [78]:
'''
class C:
    def main():
        j = 0
        for i in range(152671):        
            link = f'https://www.povarenok.ru/recipes/show/{i}/'
            html = CrawlerPovar(link).get_html()
            j = ParserPovar(html).get_info(link, j)
            if j > 25000:
                return
        '''
            

"\nclass C:\n    def main():\n        j = 0\n        for i in range(152671):        \n            link = f'https://www.povarenok.ru/recipes/show/{i}/'\n            html = CrawlerPovar(link).get_html()\n            j = ParserPovar(html).get_info(link, j)\n            if j > 25000:\n                return\n        "

In [79]:
class D:
    def mai():
        crawler = CrawlerPovar()
        links = DBHelper().print_info('link')
        parser = ParserPovar()
        flag = True
        i1 = 30014
        for i in range(32050, 55000, 1):
            flag = True
            link = f'https://www.povarenok.ru/recipes/show/{i1}/'
            flag = parser.get_info(crawler.get_html(link), i, link)
            i1 += 1
        #link = f'https://www.povarenok.ru/recipes/show/20020/'
        #html = CrawlerPovar(link).get_html()
        #ParserPovar().get_info(html, '20020')

In [80]:
#links = DBHelper().print_info('link')
#len(links)

In [81]:
f = D.mai()

In [ ]:
#a = C.main()

In [3]:
helper = DBHelper("recipe.db")

helper.generic_create_table('user',
                    [('id', 'INTEGER')],
                    None,
                    None,
                    ['username'],
                    ['password', 'tokens', 'role', 'creation_date', 'last_recipe_id'],
                    True)

helper.generic_create_table('recipe_cooked',
                    [('id', 'INTEGER')],
                    ['user_id'],
                    [('user', 'id')],
                    None,
                    ['recipe_id', 'rating'],
                    True)

helper.generic_create_table('recipe_like',
                    [('id', 'INTEGER')],
                    ['user_id'],
                    [('user', 'id')],
                    None,
                    ['recipe_id'],
                    True)

helper.generic_create_table('selected_ingredients',
                    [('id', 'INTEGER')],
                    ['user_id'],
                    [('user', 'id')],
                    None,
                    ['ingredient'],
                    True)

helper.generic_create_table('selected_fridge_ingredients',
                    [('id', 'INTEGER')],
                    ['user_id'],
                    [('user', 'id')],
                    None,
                    ['ingredient'],
                    True)

helper.generic_create_table('selected_categories',
                    [('id', 'INTEGER')],
                    ['user_id'],
                    [('user', 'id')],
                    None,
                    ['category'],
                    True)


helper.generic_create_table('folder',
                    [('id', 'INTEGER')],
                    ['user_id'],
                    [('user', 'id')],
                    None,
                    ['name', 'favorite'],
                    True)

helper.generic_create_table('recipe_folder',
                    [('id', 'INTEGER')],
                    ['folder_id'],
                    [('folder', 'id')],
                    None,
                    ['recipe_id'],
                    True)

helper.get_unique('recipe_folder', 'unique_folder_recipe',
                  ['folder_id', 'recipe_id'])
                    

CREATE TABLE IF NOT EXISTS user (id INTEGER PRIMARY KEY AUTOINCREMENT ,  username TEXT, password TEXT, tokens TEXT, role TEXT, creation_date TEXT, last_recipe_id TEXT, UNIQUE (username))
CREATE TABLE IF NOT EXISTS recipe_cooked (id INTEGER PRIMARY KEY AUTOINCREMENT ,  user_id TEXT, recipe_id TEXT, rating TEXT, FOREIGN KEY (user_id) REFERENCES user(id))
CREATE TABLE IF NOT EXISTS recipe_like (id INTEGER PRIMARY KEY AUTOINCREMENT ,  user_id TEXT, recipe_id TEXT, FOREIGN KEY (user_id) REFERENCES user(id))
CREATE TABLE IF NOT EXISTS selected_ingredients (id INTEGER PRIMARY KEY AUTOINCREMENT ,  user_id TEXT, ingredient TEXT, FOREIGN KEY (user_id) REFERENCES user(id))
CREATE TABLE IF NOT EXISTS selected_fridge_ingredients (id INTEGER PRIMARY KEY AUTOINCREMENT ,  user_id TEXT, ingredient TEXT, FOREIGN KEY (user_id) REFERENCES user(id))
CREATE TABLE IF NOT EXISTS selected_categories (id INTEGER PRIMARY KEY AUTOINCREMENT ,  user_id TEXT, category TEXT, FOREIGN KEY (user_id) REFERENCES user(id))

In [11]:
helper = DBHelper()
i = 25
cat = helper.get('recipe_categories', ['recipe_id'], [str(i)])
for c in cat:
    name = helper.get('category', ['id'], [c[2]])
    print(name[0][1])

Супы
Арабская кухня
Томатный суп


In [13]:
helper = DBHelper()
a = 25
ing = helper.get('recipe_ingredients', ['recipe_id'], [str(a)])
for i in ing:
    name = helper.get('ingredient', ['id'], [i[2]])
    print(name[0][1], i[3])

Красный лук 1 головка
Соль по вкусу
Молотый черный перец по вкусу
Оливковое масло 50 мл
Корица ¼ чайные ложки
Лимон 1 штука
Помидоры 1 кг
Петрушка 50 г
Куриный бульон 1 л
Кинза 50 г
Мед 2 чайные ложки
Имбирь 20 г
Тмин ¼ чайные ложки
Паприка 1 чайная ложка


In [ ]:
#DBHelper("recipe.db").simple_create_table("link", "id", ["link", "photo"])

In [7]:
helper = DBHelper("test.db")